In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
            .builder \
            .appName("Spark Catalog Demo") \
            .master("spark://localhost:7077") \
            .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-4.0.1-bin-hadoop3/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/spark/.ivy2.5.2/cache
The jars for the packages stored in: /home/spark/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.spark#spark-connect_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e2f14e1f-eca8-4610-9517-5b15ce3a0bce;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.1 in central
	found io.delta#delta-storage;4.0.1 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	found org.apache.spark#spark-connect_2.13;4.1.1 in central
	found org.apache.spark#spark-pipelines_2.13;4.1.1 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0 in central
	found jakarta.servlet#jakarta.servlet-api;5.0.0 in central
	found javax.servlet#javax.servlet-api;4.0.1 in central
	found com.google.guava#guava;33.4.8-jre in central
	fo

In [ ]:
print("Databases:")
spark.catalog.listDatabases()

In [ ]:


# 2. Create a Managed Table and an External Table
spark.sql("CREATE TABLE IF NOT EXISTS managed_test (id INT, data STRING)")
spark.sql("CREATE TABLE IF NOT EXISTS external_test (id INT) LOCATION '/tmp/ext_data'")

# 3. Use the Catalog to list tables and identify their types
tables = spark.catalog.listTables()
for t in tables:
    print(f"Table Name: {t.name} | Is Temporary: {t.isTemporary} | Table Type: {t.tableType}")

In [ ]:


# 4. Cache a table and check its status via the Catalog
spark.catalog.cacheTable("managed_test")
is_cached = spark.catalog.isCached("managed_test")
print(f"Is 'managed_test' cached in memory? {is_cached}")

# 5. Refresh metadata (crucial for S3/HDFS manual file updates)
# This invalidates the cache and re-scans the file system
spark.catalog.refreshTable("external_test")

# 6. Uncache to free up cluster resources
spark.catalog.uncacheTable("managed_test")